In [3]:
import pandas as pd

ModuleNotFoundError: No module named 'pandas'

In [ ]:
import os
import json
file_path = 'documents/IS/metadata/metadata.json'

with open(file_path, 'r') as f:
	data = json.load(f)

# If data is a dict, convert to DataFrame by wrapping in a list (each key becomes a column)
if isinstance(data, dict):
	df = pd.DataFrame([data])
else:
	df = pd.DataFrame(data)


In [ ]:
df

,chunks,chunk_doc_names,file_hashes
0,[Structured query language (SQL) is a programm...,[sql.txt],{'sql.txt': 'ab68be149872d34328b015251af6c9a75...


In [ ]:
from serpapi import GoogleSearch

params = {
  "engine": "google",
  "q": "Coffee",
  "api_key": "a0a3fc1d1834f2a1606f7424c5e20423a62a5fc77dfb421c10de213e461ce42c"
}

search = GoogleSearch(params)
results = search.get_dict()
organic_results = results["organic_results"]

In [ ]:
organic_results

[{'position': 1,
  'title': 'Coffee',
  'link': 'https://en.wikipedia.org/wiki/Coffee',
  'redirect_link': 'https://www.google.com/url?sa=t&source=web&rct=j&opi=89978449&url=https://en.wikipedia.org/wiki/Coffee&ved=2ahUKEwjs9NOwl5aPAxVMC0QIHb8lGSgQFnoECFIQAQ',
  'displayed_link': 'https://en.wikipedia.org › wiki › Coffee',
  'thumbnail': 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSQpT6fDhI5wR4W-N7BA0y8bAYztA0E2Yt62IcMQu-0n2KO5lQ8fLzB&usqp=CAE&s',
  'favicon': 'https://serpapi.com/searches/68a4111c529e4beabc5d4588/images/b7cc67a2b78dbb467fd73da3a5c83de36f73d07eaae80d5d69dc3d3add0d9f2d.png',
  'snippet': 'Coffee is a beverage brewed from roasted, ground coffee beans. Darkly colored, bitter, and slightly acidic, coffee has a stimulating effect on humans',
  'snippet_highlighted_words': ['a beverage brewed from roasted, ground coffee beans'],
  'sitelinks': {'inline': [{'title': 'History',
     'link': 'https://en.wikipedia.org/wiki/History_of_coffee'},
    {'title': 'Coffee pr

In [ ]:
organic_results[0]['source']

'Wikipedia'

In [ ]:
len(organic_results)

10

In [ ]:
for i in range(len(organic_results)):
    if organic_results[i]['source'] == 'Wikipedia':
        wiki_link = organic_results[i]['link']
wiki_link

'https://en.wikipedia.org/wiki/Coffee'

In [ ]:
import requests
r = requests.get(wiki_link)
data = r.text
data

'<!DOCTYPE html>\n<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-custom-font-size-clientpref-1 vector-feature-appearance-pinned-clientpref-1 vector-feature-night-mode-enabled skin-theme-clientpref-day vector-sticky-header-enabled vector-toc-available" lang="en" dir="ltr">\n<head>\n<meta charset="UTF-8">\n<title>Coffee - Wikipedia</title>\n<script>(function(){var className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-featur

In [ ]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(data, 'html.parser')
print(soup.prettify())

<!DOCTYPE html>
<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-custom-font-size-clientpref-1 vector-feature-appearance-pinned-clientpref-1 vector-feature-night-mode-enabled skin-theme-clientpref-day vector-sticky-header-enabled vector-toc-available" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   Coffee - Wikipedia
  </title>
  <script>
   (function(){var className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled 

In [ ]:
def extract_important_sections(url):
    import requests
    from bs4 import BeautifulSoup
    
    html = requests.get(url).text
    soup = BeautifulSoup(html, "html.parser")

    # Define rules 
    skip_sections = {"see also", "references", "further reading", "external links", "notes"}
    keep_keywords = {"history", "etymology", "biology", "culture"}


    content = {}

    # Extract introduction 
    intro_paragraphs = []
    for p in soup.select("p"):
        if p.find_previous("h2"):
            break
        intro_paragraphs.append(p.get_text(strip=True))
    if intro_paragraphs:
        content["Introduction"] = " ".join(intro_paragraphs)

    # Loop through h2 sections
    for h2 in soup.find_all("h2"):
        section_title = h2.get_text(strip=True)
        section_key = section_title.lower()

        # Skip unwanted sections
        if any(skip in section_key for skip in skip_sections):
            continue

        # Keep only sections with relevant keywords
        if not any(kw in section_key for kw in keep_keywords):
            continue

        # Collect paragraphs until next h2
        paragraphs = []
        for elem in h2.next_elements:
            if elem.name == "h2":
                break
            if elem.name == "p":
                paragraphs.append(elem.get_text(strip=True))

        if paragraphs:
            content[section_title] = " ".join(paragraphs)

    return content


In [ ]:
wiki_data = extract_important_sections('https://en.wikipedia.org/wiki/Coffee')

In [ ]:
import json
wiki_data = json.dumps(wiki_data)

In [ ]:
from openai import OpenAI
from dotenv import load_dotenv
import os

load_dotenv()
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
client = OpenAI(api_key=OPENAI_API_KEY)


response = client.chat.completions.create(
    model = "gpt-5-mini",
    messages=[
        {"role": "system", "content": "As a helpful study guide, summerize given wiki page for refering as an extra resource. Keep your repsonse "},
        {"role": "user", "content": wiki_data}
    ]
)

In [ ]:
wiki_summery = response.choices[0].message.content

In [ ]:
wiki_summery

'Brief study-guide summary of the provided coffee article (key facts and takeaways)\n\nEtymology\n- English "coffee" entered in 1582 from Dutch koffie ← Ottoman Turkish kahve ← Arabic qahwah.\n- qahwah in medieval Arabic was associated with "wine" or "the dark one" (referring to the beverage); Arabic name for the bean is bunn.\n- Common English phrases: "coffee pot" (from 1705) and "coffee break" (coined 1952 ad campaign).\n\nHistory — origins and global spread\n- Legendary origins: Kaldi the Ethiopian goatherd (9th century) and Sheikh Omar—both likely apocryphal; these stories appear much later in sources.\n- Earliest credible references: mid-15th century Yemen (coffee seeds roasted and brewed); used by Sufi groups to remain awake during rituals.\n- Ethiopia → Yemen via Red Sea trade; Somalia (Berbera/Zeila) and regions around Harar were important early sources.\n- 16th century: coffee spreads across the Middle East and North Africa.\n- Key individuals/events in diffusion:\n  - Baba B